# Fundamental Data Validation
Compare and validate annual financial statement numbers from different data sources

In [1]:
library(XML)
library(rlist)
library(dplyr)
library(reshape2)
library(XLConnect)
library(data.table)
setwd("/Users/jtan/Documents/Investment/Fundamental-Analysis/Data-Validation/")

# please use one symbol for now
STOCK_LIST = c('NVDA')

# avoid scientific number format
options(scipen=999)

output_table <- data.frame (
    symbol=character(),
    source=character(),
    period=character(),
    metric=character(),
    value=numeric(),
    stringsAsFactors=FALSE
)

ERROR: Error in library(rlist): there is no package called ‘rlist’


## Morningstar
[Morningstar Finalcial Report API](https://gist.github.com/hahnicity/45323026693cdde6a116)

In [7]:
# period: 12 = annual reporting, 3 = quarterly reporting
# report_type: is = Income Statement, cf = Cash Flow, bs = Balance Sheet
get_morningstar_data <- function(symbol, report_type, period) {
    # http://financials.morningstar.com/ajax/ReportProcess4CSV.html?&t=NVDA&reportType=is&period=3&dataType=A&order=asc&columnYear=5&number=1
    url <- paste('http://financials.morningstar.com/ajax/ReportProcess4CSV.html?&t=', symbol, '&reportType=', report_type, '&period=', period, '&dataType=A&order=asc&columnYear=5&number=3', sep = '')
    read.table(url, header=FALSE, fill=TRUE, na.strings=c("","NA"), sep=",", quote="\"")    
}

reshape_morningstar_data <- function(data, symbol, source, metric_array) {
    # remove the last column (TTM)
    data$V7 <- NULL
    # remove header and use fiscal year end dates as column names
    data = data[-1,]
    colnames(data) <- as.character(unlist(data[1,]))
    colnames(data)[1] <- "metric"
    data = data[-1,]
    # selecting the desired metric and remove duplicates
    data <- filter(data, data$metric %in% metric_array) 
    data <- data[!duplicated(data$metric),]
    # reshape and organize data
    data <- melt(data, id=c("metric"))
    colnames(data)[2] <- 'period'
    # only applied to income statement metric
    data$metric <- as.character(gsub("Basic", "EPS", data$metric))
    data$symbol <- symbol
    data$source <- source
    data <- data[c('symbol','source','period','metric','value')]
}

# table initialization
morningstar_table <- output_table

for (i in 1:length(STOCK_LIST)) {
    # Income Statement: Revenue, Gross Profit, Operating Income, Net Income, EPS, EBITDA
    data.is <- get_morningstar_data(STOCK_LIST[i], 'is', '12')
    result.is <- reshape_morningstar_data(data.is, STOCK_LIST[i], 'Morningstar', c('Revenue', 'Gross profit', 'Operating income', 'Net income', 'EBITDA', 'Basic'))
    morningstar_table <- rbind(result.is, morningstar_table)
    
    # Cash Flow: Operating Cash Flow, Free Cash Flow
    data.cf <- get_morningstar_data(STOCK_LIST[i], 'cf', '12')
    result.cf <- reshape_morningstar_data(data.cf, STOCK_LIST[i], 'Morningstar', c('Operating cash flow', 'Free cash flow'))
    morningstar_table <- rbind(result.cf, morningstar_table)
}

Warning message:
“attributes are not identical across measure variables; they will be dropped”Warning message:
“attributes are not identical across measure variables; they will be dropped”

## ADVFN
[Get Fundamental Data from ADVFN](https://github.com/systematicinvestor/SIT/blob/master/R/fundamental.data.r)

In [7]:
# period: annual_reports, quarterly_reports
# value: number of records available 
# symbol: may need to append NYSE or NASDAQ labels
get_advfn_data <- function(symbol, source, period, value) {
    
    table.temp <- output_table
    
    repeat {
        # http://uk.advfn.com/p.php?pid=financials&btn=quarterly_reports&symbol=NVDA&istart_date=100
        url <- paste('http://uk.advfn.com/p.php?pid=financials&btn=', period, '&symbol=NYSE:', symbol, '&istart_date=', value, sep="")
        tables <- readHTMLTable(url)
        tables <- list.clean(tables, fun = is.null, recursive = FALSE)
        n.rows <- unlist(lapply(tables, function(t) dim(t)[1]))
        data <- tables[[which.max(n.rows)]]
        
        # selecting the desired metric
        # Income Statement: Revenue, Gross Profit, Operating Income, Net Income, EPS, EBITDA
        # Cash Flow: Operating Cash Flow, Free Cash Flow
        if(period == 'quarter end date') {
            toMatch <- c('quarter end date', 'total revenue', 'gross operating profit', 'operating income', 'total net income', 'Basic EPS - Total', 'EBITDA', 'net cash from total operating activities', 'free cash flow')            
        } else {
            toMatch <- c('year end date', 'total revenue', 'gross operating profit', 'operating income', 'total net income', 'Basic EPS - Total', 'EBITDA', 'net cash from total operating activities', 'free cash flow')
        }                 
        data <- filter(data, data$V1 %in% toMatch)
        data[is.na(data)] <- 0
            
        if(nrow(data) > 0) {
            # remove empty columns
            data <- data[, sapply(data, function(col) nlevels(col) > 1)]

            # reshape and organize data
            colnames(data) <- as.character(unlist(data[1,]))
            colnames(data)[1] <- "metric"
            data = data[-1,]

            final <- melt(data, id=c("metric"))
            colnames(final)[2] <- 'period'
            final$symbol <- symbol
            final$source <- source
            final$value <- as.numeric(gsub(",", "", final$value, fixed = TRUE))
            i <- sapply(final, is.factor)
            final[i] <- lapply(final[i], as.character)
            final$period <- gsub("/", "-", final$period) 
            final <- final[c('symbol', 'source', 'period','metric','value')]

            # only add not already present data
            data <- anti_join(final, table.temp)
            if(nrow(data) > 0) {
                table.temp = rbind(data, table.temp)
            }
        }

        if(value > 0) {
            # can only get 5 time periods at a time
            value = value - 5
            value = max(0, value)
        } else
            break
    }
    table.temp
}
         
# table initialization
advfn_table <- output_table
                    
for (i in 1:length(STOCK_LIST)) {
    result <- get_advfn_data(STOCK_LIST[i], 'ADVFN', 'annual_reports', 100)
    advfn_table <- rbind(result, advfn_table)
}

# retain only the last 5 reproting periods
#truncated <- morningstar_table %>% distinct(morningstar_table$period) %>% mutate_if(is.factor, as.character)
#colnames(truncated)[1] <- 'period'
#advfn_table <- merge(truncated, advfn_table, by='period')                        
#advfn_table <- advfn_table[c('symbol','source','period','metric','value')]

## Stockrow

In [ ]:
# period: Y = annual reporting, Q = quarterly reporting
# report_type: Income Statement, Cash Flow, Balance Sheet
get_stockrow_data <- function(symbol, report_type, period) {
    # http://stockrow.com/api/companies/NVDA/financials.xlsx?dimension=MRQ&section=Income Statement
    url <- paste('http://stockrow.com/api/companies/', symbol, '/financials.xlsx?dimension=MR', period, '&section=', report_type, sep = '')
    download.file(url, 'temp.xlsx', mode="wb")
    readWorksheet(loadWorkbook('temp.xlsx'), sheet = symbol, header = TRUE)
}

reshape_stockrow_data <- function(data, symbol, source, metric_array) {
    # selecting the desired metric
    colnames(data)[1] <- 'metric'
    data <- filter(data, data$metric %in% metric_array) 
    # fix column names
    name_array <- colnames(data)
    for (i in 2:length(name_array)) { 
        name_array[i] <- substr(colnames(data)[i], 2, 8) 
    }
    name_array <- gsub("\\.", "\\-", name_array)
    colnames(data) <- name_array
    # reshape and organize data
    data <- melt(data, id=c("metric"))
    colnames(data)[2] <- 'period'
    data$symbol <- symbol
    data$source <- source
    data$value <- ifelse(abs(data$value) > 1000, as.integer(data$value/1000000), data$value)
    data <- data[c('symbol','source','period','metric','value')]
}

# table initialization
stockrow_table <- output_table

for (i in 1:length(STOCK_LIST)) {
    # Income Statement: Revenue, Gross Profit, Operating Income, Net Income, EPS, EBITDA
    data.is <- get_stockrow_data(STOCK_LIST[i], 'Income Statement', 'Y')
    result.is <- reshape_stockrow_data(data.is, STOCK_LIST[i], 'Stockrow', c('Revenues', 'Gross Profit', 'Operating Income', 'Net Income', 'EPS', 'EBITDA'))
    stockrow_table <- rbind(result.is, stockrow_table)
    
    # Cash Flow: Operating Cash Flow, Free Cash Flow
    data.cf <- get_stockrow_data(STOCK_LIST[i], 'Cash Flow', 'Y')
    result.cf <- reshape_stockrow_data(data.cf, STOCK_LIST[i], 'Stockrow', c('Operating Cash Flow', 'Free Cash Flow'))
    stockrow_table <- rbind(result.cf, stockrow_table)
    
    # Clean up
    if (file.exists('temp.xlsx')) file.remove('temp.xlsx')
}

# retain only the last 5 reproting periods
#truncated <- morningstar_table %>% distinct(morningstar_table$period) %>% mutate_if(is.factor, as.character)
#colnames(truncated)[1] <- 'period'
#stockrow_table <- merge(truncated, stockrow_table, by='period')                        
#stockrow_table <- stockrow_table[c('symbol','source','period','metric','value')]

# Final Results

In [ ]:
final_result <- do.call(rbind, list(morningstar_table, advfn_table, stockrow_table))
write.table(final_result, file = 'Financial_Metric_Annual.csv', sep = ",", col.names = TRUE, row.names = FALSE, quote = FALSE)